In [1]:
!pip install xgboost tensorflow pandas numpy scikit-learn matplotlib

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load dataset
df = pd.read_csv('/content/complete_solar_data2.csv')

# Convert DATE_TIME to datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce')

# Extract time-based features

df['MONTH'] = df['DATE_TIME'].dt.month
df['DAY'] = df['DATE_TIME'].dt.day
df['HOUR'] = df['DATE_TIME'].dt.hour

# Drop DATE_TIME column
df.drop(columns=['DATE_TIME'], inplace=True)

# Drop NaN values (if any)
df.dropna(inplace=True)

# Define target column
target_column = 'AC_POWER'

# Separate features and target
features = df.drop(columns=[target_column])
target = df[target_column]

# Feature Scaling
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Reshape data for LSTM (Sliding Window Technique)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build Improved LSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2]))),
    BatchNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(32, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu')  # Output layer
])

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train Model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr])

# Predictions
y_pred = model.predict(X_test)

# Performance Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate Accuracy using MAPE
mape = np.mean(np.abs((y_test - y_pred.flatten()) / y_test)) * 100
accuracy = 100 - mape  # Accuracy formula

# Print Results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")
print(f"Model Accuracy (100 - MAPE): {accuracy:.2f}%")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


847/847 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - loss: 95282.6172 - val_loss: 17035.2129 - learning_rate: 0.0010
Epoch 2/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - loss: 23346.2461 - val_loss: 14631.3184 - learning_rate: 0.0010
Epoch 3/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 21994.9629 - val_loss: 14210.5732 - learning_rate: 0.0010
Epoch 4/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - loss: 20746.3633 - val_loss: 15878.7627 - learning_rate: 0.0010
Epoch 5/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 22s 26ms/step - loss: 20674.8691 - val_loss: 13748.2832 - learning_rate: 0.0010
Epoch 6/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 35s 19ms/step - loss: 19652.6992 - val_loss: 14536.7383 - learning_rate: 0.0010
Epoch 7/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - loss: 19461.8613 - val_loss: 13671.4326 - learning_rate: 0.0010
Epoch 8/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - loss: 18972.1602 - val_loss: 15582.0498 - learning_rate: 0.0010
Epoch 9/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 20s

In [5]:
import pickle

# Save the trained model
model.save("solar_energy_model.h5")  # Save in HDF5 format

# Save the scaler
with open("brajesh update lstm for solar.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully!")


Model and scaler saved successfully!
